# Molecular descriptors
***

This notebook describes the process of generating a table with descriptors for the molecules in the training dataset

**Contents**

1. [Replacing SDF files](#Replacing-SDF-files)
1. [Calculating descriptors with Mordred](#Calculating-descriptors-with-Mordred)
    1. [Fixing data types and missing values](#Fixing-data-types-and-missing-values)
        1. [Issue that was not considered during the project](#Issue-that-was-not-considered-during-the-project)
        1. [Counting errors](#Counting-errors)
        1. [Converting all data types to float](#Converting-all-data-types-to-float)
    

The python library Mordred (*1) was used to calculate a series of molecular descriptors to represent the molecules in the dataset.

*1. Moriwaki, H., Tian, YS., Kawashita, N. et al. Mordred: a molecular descriptor calculator. J Cheminform 10, 4 (2018). https://doi.org/10.1186/s13321-018-0258-y*

## Replacing SDF files

Many ligand files caused errors when attempting to calculate the descriptors, and it was determined that the coordinate system was the problem, so the SDF files that were included in the PDBbind dataset were replaced with SDF files with 'ideal' coordinates from the Protein Data Bank. 

The following code retrieves from the Protein Data Bank (using their file download service - https://www.rcsb.org/docs/programmatic-access/file-download-services) the ligand files that were identified as faulty in the first attempt at using Mordred

In [1]:
import argparse
import json
import requests

def get_sdf(id_list, dest, url='https://files.rcsb.org/ligands/download/'):
    
    '''given a list of PDB ligand IDs, the function will 
    download the files to a specified destination folder'''
    
    for name in id_list:
        # making the url which contains the ID of the 
        # ligand to download, then sending a request
        url_create = '{}{}_ideal.sdf'.format(url, name)
        r = requests.get(url_create)
        result = r.text
        
        # writing the file to the destination folder
        with open('{}{}.sdf'.format(dest, name), 'w') as f:
            f.write(result)

Loading the list of new ligands:

In [2]:
new_lig_list = []

with open("C:\\Users\\Ieremita Emanuel\\Desktop\\CS_project\\uniq_lig_list_fixed.txt") as f:
    for line in f:
        new_lig_list.append(line.strip())

In [3]:
len(new_lig_list)

6824

There are 6825 ligands in the list, but some of them (19) returned http 404 errors when downloading, so they were removed, bringing down the count to 6805

In [4]:
# calling the function on the list
get_sdf(new_lig_list, "C:\\Users\\Ieremita Emanuel\\Desktop\\CS_project\\new_sdf\\")

## Calculating descriptors with Mordred

Mordred relies on the computational chemistry package rdkit (https://rdkit.org/), and to calculate the molecular descriptors it requires an rdkit 'molecule' object to be made from an SDF file, which is then can be fed to a Mordred 'Calculator' object that will compute the desired molecular descriptors (that are selected when building the object)

In [5]:
from mordred import Calculator, descriptors 
import os
import pandas as pd
from rdkit import Chem

# making mordred Calculator object with selected descriptors
calc = Calculator([descriptors.AcidBase, 
                   descriptors.Aromatic, 
                   descriptors.AtomCount, 
                   descriptors.BalabanJ, 
                   descriptors.BertzCT,  
                   descriptors.BondCount,
                   descriptors.CPSA, 
                   descriptors.EccentricConnectivityIndex, 
                   descriptors.FragmentComplexity, 
                   descriptors.GeometricalIndex, 
                   descriptors.GravitationalIndex, 
                   descriptors.HydrogenBond, 
                   descriptors.Lipinski, 
                   descriptors.McGowanVolume, 
                   descriptors.Polarizability, 
                   descriptors.RotatableBond, 
                   descriptors.SLogP, 
                   descriptors.TopoPSA.TopoPSA, 
                   descriptors.TopologicalIndex, 
                   descriptors.VdwVolumeABC, 
                   descriptors.Weight, 
                   descriptors.WienerIndex.WienerIndex
                   ])

The Calculator object has a `.pandas()` method that will convert a list of rdkit molecule objects to a Pandas DataFrame. 

We will need the ordered list of the molecules in the dataset, and then run through all of them, pick the corresponding SDF file from the directory, and calculate molecular descriptors. 

In [6]:
# importing .csv file with the PDB IDs and ligand IDs from the final selection
lig_ids_df = pd.read_csv("C:\\Users\\Ieremita Emanuel\\Desktop\\CS_project\\CSVs\\ligands_and_id.csv")

# making it a list while stripping the '()' surrounding the ligand names
lig_ids = [lig.strip('()') for lig in lig_ids_df['ligand_name'].to_list()]

# making a generator that will yield an rdkit molecule object from a directory 
# of .sdf files, in the order of the IDs in the given list
def mol_generator(folder, id_list):
    for ID in id_list:
        yield Chem.MolFromMolFile("{}{}.sdf".format(folder, ID))

In [7]:
# making the Pandas DataFrame with all the molecules in the dataset
mol_df = calc.pandas(mol_generator("C:\\Users\\Ieremita Emanuel\\Desktop\\CS_project\\new_sdf\\", lig_ids))

9607it [06:51, 23.33it/s]


,nAcid,nBase,nAromAtom,nAromBond,nAtom,nHeavyAtom,nSpiro,nBridgehead,nHetero,nH,...,TopoPSA,Diameter,Radius,TopoShapeIndex,PetitjeanIndex,Vabc,MW,AMW,WPath,WPol
0,2,1,12,12,60,33,0,0,10,27,...,184.12,14,7,1.0,0.5,437.72728,473.162057,7.886034,3528,45
1,1,0,18,18,42,28,0,0,10,14,...,149.69,16,8,1.0,0.5,327.560969,398.068491,9.477821,2428,42
2,4,0,9,10,61,39,0,0,24,22,...,299.33,19,10,0.9,0.473684,442.799549,633.041394,10.377728,5667,68
3,0,0,6,6,24,10,0,0,0,14,...,0.00,6,3,1.0,0.5,150.350475,134.109550,5.587898,126,9
4,0,0,9,10,16,9,0,0,1,7,...,15.79,4,3,0.333333,0.25,100.862348,117.057849,7.316116,79,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9602,2,0,12,12,57,32,0,0,11,25,...,161.07,19,10,0.9,0.473684,413.673691,464.134852,8.142717,3615,42
9603,2,0,9,10,37,23,0,0,13,14,...,195.88,11,6,0.833333,0.454545,248.443744,347.063084,9.380083,1191,36
9604,0,0,12,12,48,27,0,0,8,21,...,110.31,14,7,1.0,0.5,342.102736,391.108958,8.148103,1922,45
9605,0,0,24,24,106,54,0,0,11,52,...,155.09,24,12,1.0,0.5,722.593276,736.383600,6.947015,12829,78


### Fixing data types and missing values

The resulting data from Mordred has columns with integer, float, boolean and string type values. This is not ideal for using with machine learning algorithms, so they will be converted to float type in the fllowing manner:
* boolean values will be converted to 0.0 for `False` values and 1.0 for `True` values
* integers will simply be converted to float
* strings are in the data because Mordred encountered an error and wrote the error in the field (example below). The errors will be replaced with the average value in the column, and the other values simply converted to float

From the below output it can be seen that there are 51 columns with string (object) values,  38 with integers, 16 with float, and 2 with bool type values.

In [8]:
mol_df.dtypes.value_counts()

object     51
int64      38
float64    16
bool        2
dtype: int64

Below is an example of an error as a value in the data

In [9]:
mol_df.loc[1780, 'PNSA1']

"missing gasteiger charge for ['O', 'H', 'N', 'C', 'Cu'] (PNSA1/AtomicSurfaceArea/AtomicCharge/Propc)"

In [10]:
# list of column that have strings
obj_cols = [col for col in mol_df.columns if type(mol_df.loc[777, col])==str]

#### Issue that was not considered during the project

During the initial development of the project, some wrong assumptions were made about the prevalence of the errors based on a few observations on a small, biased subset of the data. I did not investigate in more detail the errors, and did not count them specifically, like I did below. Based on the assumption that they were occuring in a small number of the columns, I decided that I will just replace the errors with the average value for that column. But that was not the best choice in regards to a few columns, as it will be seen below, since almost all of the values in those columns were errors, and this of course would lead to some biased/wrong values in the data. 

However, the impact might not have been that large, since the correlation between those values and the binding affinity was likely small, so the model would learn to not weigh them much when predicting the affinity. But it nonetheless introduces unnecessary noise in the data.

One solution could have been to remove the columns that were causing issues, but because of the way Mordred works, the descriptors are within categories, so as an example, the descriptor category `Calculator.descriptors.CPSA` contains 43 separate descriptors, and if one of the descriptors was to be removed, the other 42, potentially useful, descriptors would have to be removed as well along with it.

#### Counting errors

In [11]:
# counting how many values are errors

err_count = 0
# loop through the columns in the 'obj_cols' which contains
# columns that have strings, and therefore errors
for col in obj_cols:
    # for every value in the column, check if it can be 
    # converted to a float, if otherwise it is an error that
    # will cause a ValueError, and the counter can be increased by 1
    for val in mol_df[col].to_list():
        try:
            float(val)
        except ValueError:
            err_count+=1
print("Values with an error message: \n{}".format(err_count))

Values with an error message: 
20727


There are a total of 20,727 values with an error, out of a total of 1,027,949 (9607\*107) values, which is 2% of the total values

In [12]:
# counting how many rows have at least one error in a cell

row_err_count = 0

# loop through all the rows in the data, 
# and count the errors per row in the columns from
# 'obj_cols' which is the list of columns with errors
for i in mol_df.index:
    for col in obj_cols:
        try:
            float(mol_df.loc[i, col])
        except ValueError:
            row_err_count+=1
            break
print("Rows that have at least one error: \n{}".format(row_err_count))

Rows that have at least one error: 
9595


The 20,727 values are divided across 9595 rows out of 9607 rows, so that is quite a significant number out of the total, but the average number of errors per row is 2.16

Next we will try and see the distribution of errors in the data, and try to identify the most error-prone columns

In [13]:
# dictionary to hold the error counts for each column
error_col_counts = {}

# loop through the columns in the 'obj_cols' which contains
# columns that have strings, and therefore errors
for col in obj_cols:
    # check if the value can be converted to a float, 
    # if otherwise it is an error that will cause a ValueError, 
    # and the counter can be increased by 1
    for val in mol_df[col].to_list():
        try:
            float(val)
        except ValueError:
            error_col_counts.setdefault(col, 0)
            error_col_counts[col]+=1

In [14]:
# the resulting counts displayed as a table
pd.DataFrame({'Columns': error_col_counts.keys(), 'Error counts': error_col_counts.values()})

,Columns,Error counts
0,PNSA1,33
1,PNSA2,33
2,PNSA3,33
3,PNSA4,33
4,PNSA5,35
5,PPSA1,33
6,PPSA2,33
7,PPSA3,33
8,PPSA4,33
9,PPSA5,33


From the counts we can see that there are two columns, `GRAVH` and `GRAVHp`, which have the most errors, and they likely belong to the same descriptor category and could have been removed. 

The other observation is that there are likely ~33 molecules that are getting errors consistently across the descriptors from the `CPSA` category. Those entries could also be removed since they're a small number.

One big problem with having errors come up when calculating descriptors, is how they will be dealth with when generating input data for prediction.

In that case there wouldn't be average values to replace them with, unless they are directly stored and substituted when they come up, but that is not a reliable solution since on one hand 51 values would have to be stored that contain the averages (or better, the median) from the training dataset which is somewhat unfeasable, and on the other this will not account for errors that could come up in new columns with the new data.

A more reliable solution is to try and make mordred insert a value of 0 for example instead of the error, but it is not clear how to do that yet, or how it will affect the data.

#### Converting all data types to float

As mentioned earlier, the machine learning algorithms work best with series of floats, so the values in the current molecular descriptor DataFrame need to be converted.

First we will convert the bool columns to 0.0 and 1.0, and then iterate through the other columns and try to convert the values to float, and if a ValueError is raised we know that an error in the data field was encountered which will be replaced by the average value in the column where the error is.

In [15]:
import numpy as np

# converting bool columns to float
def bool_to_bin(val):
    if val==True:
        return 1.0
    else:
        return 0.0

# applying the 'bool_to_bin()' function to all the values 
# in the 2 columns that are type bool
bool_cols = [col for col in mol_df.columns if mol_df.dtypes[col]==np.dtype('bool_')]

# applying the function to the values in the columns from 'bool_cols'
for boo in bool_cols:
    mol_df[boo] = mol_df[boo].apply(bool_to_bin)

In [16]:
mol_df.dtypes.value_counts()

object     51
int64      38
float64    18
dtype: int64

Converting the integer columns to float

In [17]:
# list of columns that are integers
ints = [col for col in mol_df.columns if mol_df.dtypes[col]==np.dtype('int64')]

In [18]:
for col in ints:
    mol_df[col] = mol_df[col].astype(float)

In [19]:
def mean_inputer(i):
    '''replaces errors with 
    the average value in the column'''
    
    summ = 0
    l = len(mol_df[i])
    for n in mol_df[i]:
        try:
            summ+=float(n)
        except ValueError:
            l-=1
            continue
    return summ/l

In [20]:
for col in obj_cols:
    for i in mol_df.index:
        try:
            mol_df.loc[i, col] = float(mol_df.loc[i, col])
        except ValueError:
            mol_df.loc[i, col] = float(mean_inputer(col))

In [21]:
mol_df.dtypes.value_counts()

float64    107
dtype: int64

All the values are converted to float and the errors dealt with, so the data can be saved into a CSV file

In [22]:
# saving the DataFrame to a .csv file to be used later in the final dataset assembly
mol_df.to_csv("C:\\Users\\Ieremita Emanuel\\Desktop\\CS_project\\CSVs\\molecules.csv", index=False)